# Exploring design_climate

This notebook walks through the exposed methods in a safe, optional-network flow.

Guidance:

- Cells are ordered for discovery: quick pure functions → optional network calls → parsers.
- Toggle flags in code cells before running any network/IO sections.


In [10]:
# Imports for exposed APIs
from fluids.design_climate import (
    IntegratedSurfaceDatabaseStation,
    StationDataGSOD,
    cooling_degree_days,
    geocode,
    get_clean_isd_history,
    get_closest_station,
    get_station_year_text,
    gsod_day_parser,
    heating_degree_days,
)
from pprint import pprint

print("Imported design_climate exposed methods")

Imported design_climate exposed methods


In [2]:
# Exposed API list
from fluids import design_climate

print(design_climate.__all__)

['IntegratedSurfaceDatabaseStation', 'StationDataGSOD', 'cooling_degree_days', 'geocode', 'get_clean_isd_history', 'get_closest_station', 'get_station_year_text', 'gsod_day_parser', 'heating_degree_days']


## Degree-day utilities (pure functions)

These are fast and safe to run with no network or disk access.


In [ ]:
# Heating/cooling degree days
print("Heating DD (303.8 K):", heating_degree_days(303.8))
print("Heating DD (273 K):", heating_degree_days(273))
print("Cooling DD (250 K):", cooling_degree_days(250))
print("Cooling DD (300 K):", cooling_degree_days(300))
print("Cooling DD (250 K, base=300 K):", cooling_degree_days(250, T_base=300))

## Geocoding (optional dependency + network)

This uses `geopy` and an online lookup. Toggle the flag before running.


In [52]:
RUN_GEOCODE = True

if RUN_GEOCODE:
    try:
        print("Geocode result:", geocode("Calgary, Alberta, Canada"))
    except Exception as exc:
        print("Geocode failed:", exc)
else:
    print("Geocoding skipped (set RUN_GEOCODE=True to enable)")

Geocode result: (51.0456064, -114.057541)


In [53]:
#getgeocode of chennai using meteostat

import meteostat as ms
from datetime import datetime

p = ms.Point(51.109, -114.000)
ms.stations.nearby(point=p, limit=1 )


,name,country,region,latitude,longitude,elevation,timezone,distance
id,,,,,,,,
DHKZ6,Calgary Int'l CS,CA,AB,51.11,-114.0,1082,America/Edmonton,111.2


In [54]:
from sympy import true


ms.daily("DHKZ6",start=datetime(2024,1,1), end=datetime(2024,12,31)).fetch(units=ms.UnitSystem.IMPERIAL, location=true,sources=true )

Column latitude is not a valid column name and won't be converted
Column longitude is not a valid column name and won't be converted
Column elevation is not a valid column name and won't be converted


,temp,temp_source,tmin,tmin_source,tmax,tmax_source,rhum,rhum_source,prcp,prcp_source,...,wpgt_source,pres,pres_source,tsun,tsun_source,cldc,cldc_source,latitude,longitude,elevation
time,,,,,,,,,,,,,,,,,,,,,
2024-01-01,31.3,eccc_daily,22.1,eccc_daily,40.5,eccc_daily,75,eccc_hourly,0.091,eccc_daily,...,None,1021.3,metno_forecast,<NA>,None,3,metno_forecast,51.11,-114.0,1082
2024-01-02,27.1,eccc_daily,17.1,eccc_daily,37.0,eccc_daily,76,eccc_hourly,0.000,eccc_daily,...,None,1022.1,metno_forecast,<NA>,None,5,metno_forecast,51.11,-114.0,1082
2024-01-03,21.0,eccc_daily,17.4,eccc_daily,24.6,eccc_daily,94,eccc_hourly,0.000,eccc_daily,...,None,1018.0,metno_forecast,<NA>,None,8,metno_forecast,51.11,-114.0,1082
2024-01-04,27.7,eccc_daily,15.3,eccc_daily,40.1,eccc_daily,75,eccc_hourly,0.000,eccc_daily,...,None,1012.6,metno_forecast,<NA>,None,3,metno_forecast,51.11,-114.0,1082
2024-01-05,29.5,eccc_daily,21.2,eccc_daily,37.6,eccc_daily,64,eccc_hourly,0.008,eccc_daily,...,eccc_daily,1014.0,metno_forecast,<NA>,None,5,metno_forecast,51.11,-114.0,1082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,31.5,eccc_daily,20.3,eccc_daily,42.8,eccc_daily,53,eccc_hourly,0.000,eccc_daily,...,eccc_daily,1000.0,metno_forecast,<NA>,None,2,metno_forecast,51.11,-114.0,1082
2024-12-28,24.6,eccc_hourly metno_forecast,18.5,eccc_hourly metno_forecast,33.8,eccc_hourly metno_forecast,70,eccc_hourly metno_forecast,0.000,eccc_hourly metno_forecast,...,None,1004.0,metno_forecast,<NA>,None,4,metno_forecast,51.11,-114.0,1082
2024-12-29,22.8,eccc_daily,11.8,eccc_daily,33.6,eccc_daily,73,eccc_hourly,0.004,eccc_daily,...,None,1009.6,metno_forecast,<NA>,None,3,metno_forecast,51.11,-114.0,1082


## Station lookup by coordinates

This queries the KD-tree built from the ISD station history data.


In [32]:
# Calgary coordinates example
try:
    station = get_closest_station(13.0836939, 80.270186, 20150000)
    pprint(station)
except Exception as exc:
    pprint("Station lookup failed:", exc)

<Weather station registered in the Integrated Surface Database, name NUMGAMBAKKAM, country IN, USAF 432780, WBAN None, coords (13.067, 80.2) Weather data from 1987 to 2024>


In [33]:
station.LAT

13.067

In [35]:
from sympy import Point


ms.stations.nearby(point=ms.Point(13.0836939, 80.270186), limit=1 )

,name,country,region,latitude,longitude,elevation,timezone,distance
id,,,,,,,,
43278,Madras,IN,TN,13.0667,80.25,6,Asia/Kolkata,2889.8


## Station data download + monthly averages

This step may download GSOD data and cache it to disk.


In [16]:
RUN_GSOD_DOWNLOAD = True

if RUN_GSOD_DOWNLOAD:
    try:
        station = get_closest_station(51.02532675, -114.049868485806, 20150000)
        station_data = StationDataGSOD(station)
        month_avg = station_data.month_average_temperature(2005, 2010)
        print("Monthly average temperatures (K):")
        pprint(month_avg)

    except Exception as exc:
        print("GSOD download or parsing failed:", exc)
else:
    print("GSOD download skipped (set RUN_GSOD_DOWNLOAD=True to enable)")

Monthly average temperatures (K):
[266.4385304659498,
 267.38199588477363,
 272.1715352449223,
 277.12321200510854,
 281.2760752688171,
 285.9193121693122,
 289.14641577060934,
 288.3139784946237,
 285.36388888888894,
 277.4990143369175,
 271.7297254150702,
 262.5424731182796]


## Fetch raw year text (optional network)

This calls `get_station_year_text` directly. It downloads and caches a single year.


In [37]:
RUN_YEAR_TEXT = True

if RUN_YEAR_TEXT:
    try:
        station = get_closest_station(51.02532675, -114.049868485806, 20150000)
        text = get_station_year_text(station.USAF, station.WBAN, 2020)
        print("Raw text length:", len(text))
        # Keep a short preview
        print(text.splitlines()[0])
    except Exception as exc:
        print("Year text fetch failed:", exc)
else:
    print("Year text fetch skipped (set RUN_YEAR_TEXT=True to enable)")

Raw text length: 51013
STN--- WBAN   YEARMODA    TEMP       DEWP      SLP        STP       VISIB      WDSP     MXSPD   GUST    MAX     MIN   PRCP   SNDP   FRSHTT


## GSOD line parser

Parse one line of GSOD data. If you have `text` from the previous cell, this will parse a real line.


In [38]:
# If you fetched year text, parse the first data line (skip header)
parsed = None

try:
    if 'text' in globals():
        lines = text.splitlines()
        if len(lines) > 1:
            parsed = gsod_day_parser(lines[1])
            print(parsed)
        else:
            print("No data lines found in text")
    else:
        print("No year text available (run the previous cell with RUN_YEAR_TEXT=True)")
except Exception as exc:
    print("Parse failed:", exc)

gsod_day(DATE=datetime.datetime(2020, 1, 1, 0, 0), TEMP=274.2611111111111, TEMP_COUNT=8, DEWP=269.03888888888895, DEWP_COUNT=8, SLP=99380.0, SLP_COUNT=8, STP=86950.0, STP_COUNT=8, VISIB=None, VISIB_COUNT=0, WDSP=3.4982222222222226, WDSP_COUNT=8, MXSPD=6.173333333333334, GUST=None, MAX=278.9277777777778, MIN=269.15000000000003, PRCP=0.0, SNDP=0.0508, FRSHTT='000000', fog=False, rain=False, snow_ice=False, hail=False, thunder=False, tornado=False)


In [59]:
from fluids import design_climate_ms
from fluids import design_climate
print("design_climate_ms exposed methods:")

design_climate_ms exposed methods:


In [ ]:
location = "Abu Dhabi"
g_code = design_climate_ms.geocode(location)
print(f"Geocode for {location}:", g_code)




In [60]:
location = "Abu Dhabi"
g_code = design_climate.geocode(location)
print(f"Geocode for {location}:", g_code)


Geocode for Abu Dhabi: (24.4538352, 54.3774014)
